In [2]:
import pandas as pd
import numpy as np
import warnings
import seaborn as sns
import plotly.graph_objects as go
from datetime import datetime
from IPython.display import display, HTML
from ipywidgets import widgets, interact_manual, Checkbox
warnings.filterwarnings('ignore')
df = pd.read_csv('https://raw.githubusercontent.com/mlmontgo/dataset/main/stock_data.csv')
rf = 0.0025
def dif(opens,close):  
    difference = close-opens
    return difference
def HPR(opens,close):
    HPR = (close-opens)/opens
    return HPR
df["Difference"] = df.apply(lambda row: dif(row['Open'],row['Close']), axis=1)
df['HPR'] = df.apply(lambda row: HPR(row['Open'],row['Close']),axis=1)

#creating seperate df for each stock
AAPL = df[df['Symbol'] == "AAPL"]
MSFT = df[df['Symbol'] == "MSFT"]
TSLA = df[df['Symbol'] == "TSLA"]
AMZN = df[df['Symbol'] == "AMZN"]

def prints(**checkboxes):
    selected_options = [option for option, value in checkboxes.items() if value]
    print("Selected options:", selected_options)

options = ['AAPL', 'AMZN', 'MSFT','TSLA']
checkboxes = {option: Checkbox(value=False, description=option) for option in options}
print("This stock analyzer will show you the candlestick plot, barplot, statistics, HPR, Sharpe ratio, and investment opinoins for your selected stocks.")
print("You may choose between 4 of the most popular stocks in the tech industry today; Apple (AAPL), Microsoft (MSFT), Amazon (AMZN), and Tesla (TSLA)")
print("To view the correlation matrix between two stocks, only select the two stocks you wish to view the correlation of.")
@interact_manual(**checkboxes)
def on_click(**checkboxes):
    columns = ['Date','Open','High','Low','Close','Adj Close','Volume','Symbol']
    stocks = pd.DataFrame(columns = columns)
    select = [option for option, value in checkboxes.items() if value]
    for i in select:
        if i == 'AAPL':
            stocks = pd.merge(stocks, AAPL, how = 'outer')
        elif i == 'MSFT':
            stocks = pd.merge(stocks,MSFT,how = 'outer')
        elif i == 'AMZN':
            stocks = pd.merge(stocks,AMZN,how = 'outer')
        elif i == 'TSLA':
            stocks = pd.merge(stocks,TSLA,how = 'outer')
        fig = sns.barplot(data = stocks, x = "Symbol", y = "Difference", estimator = 'average',errorbar = None)
        fig
    if len(select) == 2:
        numeric_columns = stocks.select_dtypes(include=np.number).columns
        correlation_matrix = stocks[numeric_columns].corr(method='pearson')
        print("Correlation Matrix:")
        print(correlation_matrix)
    mean_HPR = []
    max = 0
    max_HPR = ""
    for symbol in select:
        data = stocks[stocks['Symbol'] == symbol]
        fig = go.Figure(data=[go.Candlestick(x=data['Date'],
                                         open=data['Open'],
                                         high=data['High'],
                                         low=data['Low'],
                                         close=data['Close'])])
        fig.update_layout(title=f'{symbol} Candlestick Chart')
        fig.show()
        print("Stock Statistics: ")
    grouped_stats = stocks.groupby('Symbol')['Difference'].describe()
    for symbol in select:
        data = stocks[stocks['Symbol'] == symbol]
        mean = np.mean(data['HPR'])
        mean = round(mean,5)
        if max_HPR == 0:
            max_HPR = symbol
            max = mean
        elif mean > max:
            max_HPR = symbol
            max = mean
        mean_HPR.append(mean)
        print(f'{symbol} HPR: {mean}')
    count = 0
    maxS = 0
    max_Sharpe = ""
    for symbol in select:
        data = stocks[stocks['Symbol'] == symbol]
        Sharpe = (mean_HPR[count] - rf)/np.std(data['Difference'])
        Sharpe = round(Sharpe,5)
        print(f'{symbol} Sharpe: {Sharpe}')
        if maxS == 0:
            maxS = Sharpe
            max_Sharpe = symbol
        if Sharpe > maxS:
            max_Sharpe = symbol
            maxS = Sharpe
        count+=1
    print(grouped_stats)
    print("A Sharpe value represents a stocks return per unit of risk that it takes on. Investors typically like to invest in stocks with the highest Sharpe ratio.")
    print("The Sharpe ratio calculations are assuming a risk free rate of .25%")
    print(f'With youre chosen stocks, {max_Sharpe} has the highest Sharpe ratio of {maxS}, and therefore out of these stocks should be invested in.')
    if max_HPR == max_Sharpe:
        print(f'The same stock, {max_HPR}, also has the highest holding period return, of {max}')

This stock analyzer will show you the candlestick plot, barplot, statistics, HPR, Sharpe ratio, and investment opinoins for your selected stocks.
You may choose between 4 of the most popular stocks in the tech industry today; Apple (AAPL), Microsoft (MSFT), Amazon (AMZN), and Tesla (TSLA)
To view the correlation matrix between two stocks, only select the two stocks you wish to view the correlation of.


interactive(children=(Checkbox(value=False, description='AAPL'), Checkbox(value=False, description='AMZN'), Ch…